In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from copy import copy

In [2]:
filename = 'CRTRAITS.TXT'

data = pd.read_csv(filename, sep='\t', encoding='utf-8')
data.head()

,Singular,Plural,Wood,Mercury,Ore,Sulfur,Crystal,Gems,Gold,Fight Value,...,Attack,Defense,MinDmg,MaxDmg,Shots,Spells,GuardsLow,GuardsHigh,Ability Text,Attributes
0,Pikinier,Pikinierzy,0,0,0,0,0,0,60,100,...,4,5,1,3,0,0,20,50,Powstrzymuje szarżę.,0
1,Halabardnik,Halabardnicy,0,0,0,0,0,0,75,115,...,6,5,2,3,0,0,20,30,Powstrzymuje szarżę.,0
2,Łucznik,Łucznicy,0,0,0,0,0,0,100,115,...,6,3,2,3,12,0,16,30,NaN,SHOOTING_ARMY
3,Kusznik,Kusznicy,0,0,0,0,0,0,150,115,...,6,3,2,3,24,0,16,25,Strzela dwukrotnie.,const_two_attacks | SHOOTING_ARMY
4,Gryf,Gryfy,0,0,0,0,0,0,200,324,...,8,8,3,6,0,0,12,25,Dwukrotnie kontratakuje.,DOUBLE_WIDE | FLYING_ARMY


In [3]:
data.Attributes.fillna(u'0', inplace=True)
data.Attributes.values[data.Attributes.values == u'0'] = ''
set(' | '.join(data.Attributes.values).split(' | '))

{u'',
 u'CATAPULT',
 u'DOUBLE_WIDE',
 u'FLYING_ARMY',
 u'HAS_EXTENDED_ATTACK',
 u'IMMUNE_TO_FIRE_SPELLS',
 u'IMMUNE_TO_MIND_SPELLS',
 u'IS_UNDEAD',
 u'KING_1',
 u'KING_2',
 u'KING_3',
 u'MULTI_HEADED',
 u'SHOOTING_ARMY',
 u'SIEGE_WEAPON',
 u'const_free_attack',
 u'const_jousting',
 u'const_lowers_morale',
 u'const_no_melee_penalty',
 u'const_no_wall_penalty',
 u'const_raises_morale',
 u'const_two_attacks'}

In [4]:
# do wzięcia pod uwagę w symulacjach 1vs1
keywords = {'DOUBLE_WIDE', 
            'SHOOTING_ARMY', 
            'const_free_attack', 
            'const_jousting',
            'const_no_melee_penalty',
            'const_two_attacks'}

In [5]:
name = ['Singular']
crap = ['Plural', 'Wood', 'Mercury', 'Ore', 'Sulfur', 'Crystal', 'Gems', 'Gold', 'Ability Text']
growth = ['Growth', 'Horde Growth']
abilities = ['Attributes']
spells = ['Spells']
guards_quantity = ['GuardsLow', 'GuardsHigh']

cols_to_drop = crap + growth + guards_quantity + spells
data.drop(cols_to_drop, axis=1, inplace=True)

In [6]:
data.head()

,Singular,Fight Value,AI Value,Hit Points,Speed,Attack,Defense,MinDmg,MaxDmg,Shots,Attributes
0,Pikinier,100,80,10,4,4,5,1,3,0,
1,Halabardnik,115,115,10,5,6,5,2,3,0,
2,Łucznik,115,126,10,4,6,3,2,3,12,SHOOTING_ARMY
3,Kusznik,115,184,10,6,6,3,2,3,24,const_two_attacks | SHOOTING_ARMY
4,Gryf,324,351,25,6,8,8,3,6,0,DOUBLE_WIDE | FLYING_ARMY


In [7]:
class unit_type(object):
    def __init__(self, name, fightv, aiv, hp, spd, att, df, dmlow, dmhi, shots, abi):
        self.name = name
        self.fight_value = fightv
        self.ai_value = aiv
        self.hp = hp
        self.speed = spd
        self.attack = att
        self.defense = df
        self.dmg_min = dmlow
        self.dmg_max = dmhi
        self.shots = shots
        self.attributes = {x for x in abi.split(' | ') if x in keywords}
        
class stack(object):
    def __init__(self, unit, count):
        self.count = count
        self.unit = unit
        self.hp_left = self.unit.hp
        
    def take_dmg(self, dmg):
        if dmg < self.hp_left:
            self.hp_left -= dmg
        else:
            dmg -= self.hp_left
            num_killed, rem = divmod(dmg, self.unit.hp)
            self.count -= num_killed + 1
            self.hp_left = self.unit.hp - rem
            self.count = max(self.count, 0)
            
    def attack(self, other, dmg_bonus=0., range_or_melee_penalty=False):
        if self.count < 10:
            base_dmg = sum(np.random.randint(self.unit.dmg_min, self.unit.dmg_max+1, 
                                             size=self.count))
        else:
            base_dmg = sum(np.random.randint(self.unit.dmg_min, self.unit.dmg_max+1, 
                                             size=10)) * self.count / 10
        
        att_to_def = self.unit.attack - other.unit.defense
        dmg_reduction = 0.
        if att_to_def > 0:
            dmg_bonus += min(.05 * att_to_def, 3.)
        else:
            dmg_reduction = min(.025 * -att_to_def, .7)
            
        if self.unit.name == 'Upiorny rycerz' and np.random.rand() < .2:
            dmg_bonus += 1.
            
        damage = base_dmg * (1. + dmg_bonus) * (1. - dmg_reduction)
        if range_or_melee_penalty:
            damage /= 2
            
        other.take_dmg(int(damage))

In [8]:
def make_unit(name):
    return unit_type(*data.values[data.Singular.values == name][0])

In [9]:
def fight(stackA, stackB, num_iter):
    wins = {stackA.unit.name : 0,
            stackB.unit.name : 0}
    
    def units_order():
        temp = sorted([copy(stackA),copy(stackB)], key=lambda x: x.unit.speed, reverse=True)
        if stackA.unit.speed == stackB.unit.speed and np.random.rand() < .5:
            return reversed(temp)
        return temp
        
    current, other = units_order()
    
    print current.unit.name + ': {}'.format(current.count)
    print other.unit.name + ': {}'.format(other.count)
    print '----------------\n'

    for i in xrange(num_iter):
        while current.count > 0 and other.count > 0:
            current.attack(other)
            if other.count > 0:
                other.attack(current)
            current, other = other, current
            
        winner = current if current.count > 0 else other
        wins[winner.unit.name] += 1

#         print winner.unit.name + ': {}'.format(winner.count)
        current, other = units_order()
    return wins

In [230]:
A = stack(make_unit(u'Gigant'), 10)
B = stack(make_unit(u'Diabeł'), 11)

print A.unit.name + ': {}'.format(A.count)
print B.unit.name + ': {}\n'.format(B.count)

current, other = sorted([A,B], key=lambda x: x.unit.speed, reverse=True)
l = max(len(A.unit.name), len(B.unit.name))

while A.count > 0 and B.count > 0:
    print current.unit.name.ljust(l) + ' --- atak ---> ' + other.unit.name
    current.attack(other)
    if other.count > 0:
        print current.unit.name.ljust(l) + ' <-- kontra -- ' + other.unit.name
        other.attack(current)
    print A.unit.name + ': {}'.format(A.count)
    print B.unit.name + ': {}\n'.format(B.count)
    current, other = other, current

Gigant: 10
Diabeł: 11

Diabeł --- atak ---> Gigant
Diabeł <-- kontra -- Gigant
Gigant: 8
Diabeł: 9

Gigant --- atak ---> Diabeł
Gigant <-- kontra -- Diabeł
Gigant: 6
Diabeł: 7

Diabeł --- atak ---> Gigant
Diabeł <-- kontra -- Gigant
Gigant: 4
Diabeł: 6

Gigant --- atak ---> Diabeł
Gigant <-- kontra -- Diabeł
Gigant: 3
Diabeł: 4

Diabeł --- atak ---> Gigant
Diabeł <-- kontra -- Gigant
Gigant: 2
Diabeł: 4

Gigant --- atak ---> Diabeł
Gigant <-- kontra -- Diabeł
Gigant: 1
Diabeł: 3

Diabeł --- atak ---> Gigant
Gigant: 0
Diabeł: 3



In [74]:
A = stack(make_unit(u'Pikinier'), 835)
B = stack(make_unit(u'Minotaur'), 94)

result = fight(A, B, 2000)
result

Minotaur: 94
Pikinier: 835
----------------



{u'Minotaur': 973, u'Pikinier': 1027}

In [67]:
A = stack(make_unit(u'Pikinier'), 5019)
B = stack(make_unit(u'Anioł'), 128)

result = fight(A, B, 2000)
result

Anioł: 128
Pikinier: 5019
----------------



{u'Anio\u0142': 1083, u'Pikinier': 917}

In [58]:
A = stack(make_unit(u'Anioł'), 1057)
B = stack(make_unit(u'Minotaur'), 5019)

result = fight(A, B, 2000)
result

Anioł: 1057
Minotaur: 5019
----------------



{u'Anio\u0142': 1022, u'Minotaur': 978}